In [20]:
import json
from glob import glob
import re
import mp
import os

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

In [ ]:
files = glob('tts-filtered-chunk-rows/*.json')
len(files)

In [10]:
len(files)

4442999

In [2]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [11]:
from tqdm import tqdm

already = set()
filtered = []
for f in tqdm(files):
    with open(f) as fopen:
        d = json.load(fopen)
    t = [d_['text'] for d_ in d['alignment']]
    t = clean(' '.join(t))
    if t in already:
        continue

    filtered.append(f)
    already.add(t)

100%|██████████| 4442999/4442999 [03:22<00:00, 21892.50it/s]


In [12]:
len(filtered)

2378184

In [14]:
!mkdir tts-filtered-chunk-rows-audio-dedup

In [13]:
filtered[0]

'tts-filtered-chunk-rows/175589-1.json'

In [15]:
import shutil

# shutil.copyfile(src, dst)

In [26]:
def loop(files):
    files, _ = files
    for f in tqdm(files):
        with open(f) as fopen:
            d = json.load(fopen)
        f = d['filename_audio']
        new_f = os.path.join('tts-filtered-chunk-rows-audio-dedup', os.path.split(f)[1])
        shutil.copyfile(f, new_f)

In [27]:
mp.multiprocessing(filtered, loop, cores = 10, returned = False)

100%|██████████| 237818/237818 [01:58<00:00, 2006.80it/s]


In [29]:
data = []
for f in tqdm(filtered):
    with open(f) as fopen:
        d = json.load(fopen)
    f = d['filename_audio']
    new_f = os.path.join('tts-filtered-chunk-rows-audio-dedup', os.path.split(f)[1])
    if os.path.exists(new_f):
        d['filename_audio'] = new_f
        data.append(d)

100%|██████████| 2378184/2378184 [01:37<00:00, 24505.97it/s]


In [30]:
!pip3 install pandas pyarrow

In [31]:
import pandas as pd

In [32]:
pd.DataFrame(data).to_parquet('chunk.parquet')

In [33]:
!huggingface-cli upload mesolitica/Malaysian-TTS-v2 chunk.parquet --repo-type=dataset

Uploading...: 100%|█████████████████████████▉| 357M/358M [00:18<00:00, 19.6MB/s]
https://huggingface.co/datasets/mesolitica/Malaysian-TTS-v2/blob/main/chunk.parquet


In [34]:
data[0]

{'filename_audio': 'tts-filtered-chunk-rows-audio-dedup/175589-1.mp3',
 'speaker': 'idayu',
 'alignment': [{'end': 4.02, 'score': -1.938, 'start': 3.88, 'text': 'nama'},
  {'end': 4.18, 'score': -1.632, 'start': 4.08, 'text': 'saya'},
  {'end': 4.34, 'score': -7.421, 'start': 4.22, 'text': 'Imran'},
  {'end': 4.66, 'score': -2.791, 'start': 4.38, 'text': 'Ibaad,'}],
 'averaged_pitch': [223.581, 209.382, 191.263, 193.948],
 'distances': [0.015, 0.01, 0.008, 0.063]}